In [7]:
import os
import json
import datetime as dt
import pandas as pd
import torch
import torch.nn as nn
from pybit.unified_trading import HTTP

In [8]:
test = True
if test:
    api_key = "z4y2nnrknpHgvEPRtp"
    api_secret = "tZlx7133h1S0uTpQUfj75W5NVIKPzqaX0yuB"
else:
    api_key = "dBlgdc5AZ29fInzDGE"
    api_secret = "PMcu1fTICd5jAauLnJEU4biPCKx6PTYeuxxg"

DATA_DIR = "data"
SAMPLE_PREFIX = "samples_"
OUTPUT_DIR = os.path.join("Data to test the model")
os.makedirs(OUTPUT_DIR, exist_ok=True)
session = HTTP(api_key=api_key, api_secret=api_secret, testnet=False)

In [9]:
class ConvTCNModel(nn.Module):
    def __init__(self, num_symbols, num_features, sample_len, future_steps):
        super().__init__()
        self.extractor = nn.Sequential(
            nn.Conv1d(num_features, 32, 3, stride=2, padding=1),
            nn.ELU(),
            nn.BatchNorm1d(32),
            nn.Conv1d(32, 64, 3, stride=2, padding=1),
            nn.ELU(),
            nn.BatchNorm1d(64),
        )
        self.seq_len = sample_len // 4
        self.tcn = nn.Sequential(
            nn.Conv1d(64, 64, 3, padding=1, dilation=1),
            nn.ELU(),
            nn.BatchNorm1d(64),
            nn.Conv1d(64, 64, 3, padding=1, dilation=1),
            nn.ELU(),
            nn.BatchNorm1d(64),
        )
        self.pool = nn.AdaptiveAvgPool1d(1)
        self.fc   = nn.Sequential(
            nn.Linear(64, 128),
            nn.ELU(),
            nn.LayerNorm(128),
            nn.Linear(128, future_steps),
        )

    def forward(self, x):
        B, S, F, L = x.size()
        z = x.view(B * S, F, L)
        z = self.extractor(z)
        z = self.tcn(z)
        z = z.view(B, S, 64, self.seq_len).mean(dim=1)
        z = self.pool(z).squeeze(-1)
        return self.fc(z)


In [10]:
def load_history(directory="data") -> dict[str, pd.DataFrame]:
    data = {}
    for fn in os.listdir(directory):
        if fn.endswith("_15m_full.csv"):
            sym = fn.replace("_15m_full.csv", "")
            df  = pd.read_csv(os.path.join(directory, fn), parse_dates=["timestamp"]).set_index("timestamp")
            data[sym] = df
    return data

In [11]:
def fetch_new_data(symbol: str, since: dt.datetime) -> pd.DataFrame:
    start_ms = int((since + dt.timedelta(seconds=1)).timestamp() * 1000)
    end_ms   = int(dt.datetime.utcnow().timestamp() * 1000)
    resp = session.get_kline(
        category='linear', symbol=symbol,
        interval=15, start=start_ms, end=end_ms, limit=1000
    ).get("result", {})
    lst = resp.get("list", [])
    df  = pd.DataFrame(lst, columns=['timestamp','open','high','low','close','volume','turnover'])
    if df.empty:
        return df
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    return df.set_index('timestamp')


In [12]:
def main():
    # load shape params
    with open("BTC_model/shape_params.json") as f:
        params = json.load(f)
    S = params["S"]
    F = params["F"]
    L = params["L"]
    future_steps = params["future_steps"]

    print(f"Init model with sample_len={L}, future_steps={future_steps}")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = ConvTCNModel(S, F, L, future_steps).to(device)

    history = load_history("data")

    out_dir = "Data to test the model"
    os.makedirs(out_dir, exist_ok=True)

    for sym, df in history.items():
        print(f"\nProcessing {sym}: history rows={len(df)}")
        last_ts = df.index.max()

        # חיתוך היסטוריה
        hist_slice = df.tail(L)
        print(f"  • Hist slice rows={len(hist_slice)} from {hist_slice.index.min()}")

        # הורדת חדש
        new_df = fetch_new_data(sym, last_ts)
        if new_df.empty:
            print(f"  • No new data for {sym}, skipping")
            continue

        # הרכבה ושמירה
        test_df = pd.concat([hist_slice, new_df]).sort_index().drop_duplicates()
        print(f"  • Combined rows={len(test_df)}")
        path = os.path.join(out_dir, f"{sym}_15m_test.csv")
        test_df.to_csv(path)
        print(f"✅ Saved {path}")

if __name__ == "__main__":
    main()

Init model with sample_len=2500, future_steps=100

Processing BTCUSDT: history rows=97410
  • Hist slice rows=2500 from 2025-03-28 15:30:00


C:\Users\GameReaper\AppData\Local\Temp\ipykernel_18344\1874434021.py:12: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')


  • Combined rows=3500
✅ Saved Data to test the model\BTCUSDT_15m_test.csv

Processing LTCUSDT: history rows=97410
  • Hist slice rows=2500 from 2025-03-28 15:30:00


C:\Users\GameReaper\AppData\Local\Temp\ipykernel_18344\1874434021.py:12: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')


  • Combined rows=3500
✅ Saved Data to test the model\LTCUSDT_15m_test.csv

Processing LUNA2USDT: history rows=97410
  • Hist slice rows=2500 from 2025-03-28 15:30:00


...

C:\Users\GameReaper\AppData\Local\Temp\ipykernel_18344\1874434021.py:12: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')


  • Combined rows=3500
✅ Saved Data to test the model\TWTUSDT_15m_test.csv

Processing UMAUSDT: history rows=65608
  • Hist slice rows=2500 from 2025-03-26 18:45:00


C:\Users\GameReaper\AppData\Local\Temp\ipykernel_18344\1874434021.py:12: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')


  • Combined rows=3500
✅ Saved Data to test the model\UMAUSDT_15m_test.csv

Processing UNIUSDT: history rows=97231
  • Hist slice rows=2500 from 2025-03-26 18:45:00
  • Combined rows=3500
✅ Saved Data to test the model\UNIUSDT_15m_test.csv


C:\Users\GameReaper\AppData\Local\Temp\ipykernel_18344\1874434021.py:12: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
